In [2]:
# 공항 cctv 이상행동 감지
import cv2
import numpy as np
import matplotlib.pyplot as plt


# 모델 & 설정 파일
model = "pre-trained/yolo_v3/yolov3.weights"
config = "pre-trained/yolo_v3/yolov3.cfg"
class_labels = "pre-trained/yolo_v3/coco.names"
confThreshold = 0.5
nmsThreshold = 0.4
magThreshold = 0.1

In [3]:
# 네트워크 생성
network = cv2.dnn.readNet(model, config)

if network.empty():
    print("Network open failed!")

# 클래스 이름 불러오기
classes = []
with open(class_labels, "rt") as f:
    classes = f.read().rstrip("\n").split("\n")

# # 각각의 클래스에 대한 랜덤한 색상
# colors = np.random.uniform(0, 255, (len(classes), 3))

# 출력 레이어 이름 받아오기
output_layers = list(network.getUnconnectedOutLayersNames())
print(output_layers)

['yolo_82', 'yolo_94', 'yolo_106']


In [10]:
video = cv2.VideoCapture("dataset/alchera_cctv_data/rush/camera1_20191022_100307.avi")

if video.isOpened():
    print("Video open succeed!")

WIDTH = round(video.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = round(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(WIDTH, HEIGHT)

FPS = round(video.get(cv2.CAP_PROP_FPS))
print(FPS)
DELAY = round(1000 / FPS)
print(DELAY)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

writer = cv2.VideoWriter("videos/cctv.mp4", fourcc, FPS, (WIDTH, HEIGHT))

# 파네백 옵티컬플로우 클래스 인스턴스 생성
farneback = cv2.FarnebackOpticalFlow_create()

_, prev_frame = video.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_frame = cv2.resize(prev_frame, (0, 0), None, 1/4, 1/4, interpolation=cv2.INTER_AREA)
print(prev_frame.shape)

Video open succeed!
1920 1080
25
40
(270, 480)


In [15]:
video = cv2.VideoCapture("dataset/alchera_cctv_data/rush/camera1_20191022_100307.avi")

if video.isOpened():
    print("Video open succeed!")

WIDTH = round(video.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = round(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(WIDTH, HEIGHT)

FPS = round(video.get(cv2.CAP_PROP_FPS))
print(FPS)
DELAY = round(1000 / FPS)
print(DELAY)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

writer = cv2.VideoWriter("videos/cctv.mp4", fourcc, FPS, (WIDTH, HEIGHT))

# 파네백 옵티컬플로우 클래스 인스턴스 생성
farneback = cv2.FarnebackOpticalFlow_create()

_, prev_frame = video.read()
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_frame = cv2.resize(prev_frame, (0, 0), None, 1/4, 1/4, interpolation=cv2.INTER_AREA)

# 실행
while True:
    retval, image = video.read()
    
    if not retval:
        break
    
    next_frame = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    next_frame = cv2.resize(next_frame, (0, 0), None, 1/4, 1/4, interpolation=cv2.INTER_AREA)
    
    
    # 블롭 생성
    blob = cv2.dnn.blobFromImage(image, 1/255., (320, 320), swapRB=True)
    network.setInput(blob)
    # 추론
    outs = network.forward(output_layers)
    # outs는 3개의 ndarray 리스트.
    # outs[0].shape=(507, 85), 13*13*3=507
    # outs[1].shape=(2028, 85), 26*26*3=2028
    # outs[2].shape=(8112, 85), 52*52*3=8112
    
    HEIGHT, WIDTH = image.shape[:2]
    
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            # detection: 4(bounding box) + 1(objectness score) + 80(class confidence)
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confThreshold:
                # 바운딩 박스 중심 좌표 & 박스 크기
                cx = int(detection[0] * WIDTH)
                cy = int(detection[1] * HEIGHT)
                bw = int(detection[2] * WIDTH)
                bh = int(detection[3] * HEIGHT)
                
                # 바운딩 박스 좌상단 좌표: 중심좌표 - 가로 세로 크기의 반 = 좌상단 좌표
                x = int(cx - (bw/2))
                y = int(cy - (bh/2))
                
                boxes.append([x, y, bw, bh])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))
    
    # 비최대 억제(NMS, Non-Maximum Suppression)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    
    for i in indices:
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        cv2.rectangle(image, (x, y, w, h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        
        # 파네벡 옵티컬 플로우 계산
        flow = farneback.calc(prev_frame, next_frame, None)
        
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=True)
        _, magMax, _, _ = cv2.minMaxLoc(magnitude)
        
        if magMax > 50:
            label = "Anomaly Detected: Rush"
            cv2.putText(image, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        print("max value of magnitude:", magMax)
        print("mean value of angle:", np.mean(angle))
    
    writer.write(image)  # 동영상 저장
    
    cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("frame", 960, 540)
    cv2.imshow("frame", image)
    
    if cv2.waitKey(DELAY) == 27:
        print("Video was interrupted!")
        break

video.release()  # 할당된 자원을 해제
writer.release()    # 동영상 상자를 닫음
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!
1920 1080
25
40
max value of magnitude: 13.171359062194824
mean value of angle: 174.9965
max value of magnitude: 16.9409122467041
mean value of angle: 176.02983
max value of magnitude: 20.532424926757812
mean value of angle: 175.59961
max value of magnitude: 30.05986785888672
mean value of angle: 174.8157
max value of magnitude: 36.51573181152344
mean value of angle: 174.90584
max value of magnitude: 38.976261138916016
mean value of angle: 175.54659
max value of magnitude: 42.616539001464844
mean value of angle: 176.03662
max value of magnitude: 43.999759674072266
mean value of angle: 176.5567
max value of magnitude: 47.5898323059082
mean value of angle: 176.41829
max value of magnitude: 48.22769546508789
mean value of angle: 179.01584
max value of magnitude: 64.24024963378906
mean value of angle: 178.60219
max value of magnitude: 48.87639617919922
mean value of angle: 178.10385
max value of magnitude: 48.737579345703125
mean value of angle: 178.6204
max value of ma

-1

In [13]:
video.release()  # 할당된 자원을 해제
writer.release()    # 동영상 상자를 닫음
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [24]:
a = [0,2,3,4,5]
np.argmax(a)

4

In [19]:
frame_count

1

In [7]:
# 파네백 옵티컬플로우 계산 예제
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np


# 진행 방향을 Hue 값으로 표현
def draw_flow_as_HUE(HSV, flow):
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1], angleInDegrees=True)
    HSV[..., 0] = angle / 2
    HSV[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    BGR = cv2.cvtColor(HSV, cv2.COLOR_HSV2BGR)
    
    return BGR


# 진행 방향을 직선으로 표현
def draw_flow_as_line(next_image, flow, step=16):
    h, w = next_image.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    visualization = cv2.cvtColor(next_image, cv2.COLOR_GRAY2BGR)
    cv2.polylines(visualization, lines, False, (0, 255, 255), lineType=cv2.LINE_AA)

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(visualization, (x1, y1), 1, (0, 128, 255), -1, lineType=cv2.LINE_AA)

    return visualization


video = cv2.VideoCapture("dataset/alchera_cctv_data/rush/camera1_20191015_140659.avi")

if video.isOpened():
    print("Video open succeed!")

WIDTH = round(video.get(cv2.CAP_PROP_FRAME_WIDTH))
HEIGHT = round(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(WIDTH, HEIGHT)

FPS = round(video.get(cv2.CAP_PROP_FPS))
print(FPS)
DELAY = round(1000 / FPS)
print(DELAY)

retval, prev_frame = video.read()

if not retval:
    print("frame read failed!")

prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# prev_frame과 같은 크기 & 같은 타입의 0으로 초기화된 HSV 색 공간을 정의하고 S 평면을 255로 초기화
HSV = np.zeros_like(prev_frame)
HSV[..., 1] = 255  # Saturation = 255

# 파네백 옵티컬플로우 클래스 인스턴스 생성
farneback = cv2.FarnebackOpticalFlow_create()

# 매 프레임 처리
while True:
    retval, next_frame = video.read()
    
    if not retval:
        break
    
    next_frame_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    flow = farneback.calc(prev_frame_gray, next_frame_gray, None)
    # flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, next_frame_gray, None, 0.5, 3, 13, 10, 5, 1.1, 0)
    
    cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("frame", 480, 270)
    cv2.imshow("frame", next_frame)
    
    # 진행 방향을 Hue 값으로 표현
    cv2.namedWindow("hue flow", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("hue flow", 480, 270)
    cv2.imshow("hue flow", draw_flow_as_HUE(HSV, flow))
    
    # 진행 방향을 직선으로 표현
    cv2.namedWindow("line flow", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("line flow", 480, 270)
    cv2.imshow("line flow", draw_flow_as_line(next_frame_gray, flow))
    
    if cv2.waitKey(DELAY) == 27:
        print("Video was interrupted!")
        break
    
    prev_frame_gray = next_frame_gray

video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

Video open succeed!
1920 1080
25
40


-1